In [19]:
import sys
sys.path.append('/workspaces/asclepius/src/Api')

print(sys.path)

['/workspaces/asclepius/src/Api/notebooks', '/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/home/vscode/.local/lib/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/workspaces/asclepius/src/Api', '/workspaces/asclepius/src/Api']


In [20]:
import semantic_kernel as sk
import pandas as pd
import tiktoken

import Plugins.parse_notes_plugin as pnp

In [21]:
enc = tiktoken.get_encoding('cl100k_base')

In [22]:
note_df = pd.read_csv('/workspaces/asclepius/src/Api/data/notes_no_PHI.csv')

# calculate length of 'NOTE_TEXT' column and add value as another column
note_df['NOTE_TEXT_TOKENS'] = note_df['NOTE_TEXT'].apply(lambda x: len(enc.encode(x)))

note_df['NOTE_TEXT_TOKENS']

0    1560
1    4203
Name: NOTE_TEXT_TOKENS, dtype: int64

In [23]:
plugin = pnp.ParseNotesPlugin()

note_df['CHIEF_COMPLAINT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_chief_complaint(input=x))
note_df['CHIEF_COMPLAINT_TOKENS'] = note_df['CHIEF_COMPLAINT'].apply(lambda x: len(enc.encode(x)))

note_df['ROS'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_ros(input=x))
note_df['ROS_TOKENS'] = note_df['ROS'].apply(lambda x: len(enc.encode(x)))

note_df['ASSESSMENT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_assessment(input=x))
note_df['ASSESSMENT_TOKENS'] = note_df['ASSESSMENT'].apply(lambda x: len(enc.encode(x)))

note_df[['NOTE_TEXT_TOKENS', 'CHIEF_COMPLAINT_TOKENS', 'ROS_TOKENS', 'ASSESSMENT_TOKENS']]

#print(note_df['CHIEF_COMPLAINT'][1])


,NOTE_TEXT_TOKENS,CHIEF_COMPLAINT_TOKENS,ROS_TOKENS,ASSESSMENT_TOKENS
0,1560,481,0,447
1,4203,672,100,2624


In [22]:
print(note_df['ASSESSMENT'][1])

ASSESSMENT and PLAN: 
 

  ICD-10-CM ICD-9-CM 
1. Medicare annual wellness visit, subsequent Z00.00 V70.0 
 
2. Hypothyroidism due to acquired atrophy of thyroid E03.4 244.8 liothyronine (CYTOMEL) 5 mcg Tablet
  246.8 TSH
  T3 FREE
  T4 FREE
  levothyroxine 137 mcg tablet
  T4 FREE
  T3 FREE
  TSH
 
3. Prolactinoma D35.2 227.3 PROLACTIN
  PROLACTIN
 
4. Vitamin D deficiency E55.9 268.9 VITAMIN D 25 HYDROXY
  VITAMIN D 25 HYDROXY
 
5. Essential hypertension, benign I10 401.1 CBC WITHOUT DIFFERENTIAL
  COMPREHENSIVE METABOLIC PANEL
  LIPID PANEL
  metoprolol succinate (TOPROL XL) 50 mg Extended Release 24 hour tablet
  LIPID PANEL
  COMPREHENSIVE METABOLIC PANEL
  CBC WITHOUT DIFFERENTIAL
 
6. Palpitations R00.2 785.1 metoprolol succinate (TOPROL XL) 50 mg Extended Release 24 hour tablet
  EKG 12-LEAD
 
7. Lumbar disc disease M51.9 722.93 traMADoL (ULTRAM) 50 mg tablet
 
 
 
Labs today
Increase toprol back to 1 pill daily
Decrease cytomel to 5mcg daily, continue levothyroxine
Tramadol re

In [17]:
kernel = sk.Kernel()

plugin = kernel.import_plugin_from_prompt_directory('/workspaces/asclepius/src/Api/Plugins', 'sample_plugins')

plugin.functions



{}

In [ ]:
import asyncio
import os

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from semantic_kernel.utils.settings import (
    openai_settings_from_dot_env,
)


async def main():
    kernel = Kernel()

    api_key, _ = openai_settings_from_dot_env()

    service_id = "default"
    chat_service = OpenAIChatCompletion(
        ai_model_id="gpt-4-0613",
        service_id=service_id,
        api_key=api_key,
    )
    kernel.add_service(chat_service)

    chat_history = ChatHistory(system_message="Assistant is a large language model")

    cur_dir = os.path.join(os.path.dirname(os.path.realpath(__file__)), "resources")
    plugin = kernel.import_plugin_from_prompt_directory(cur_dir, "sample_plugins")

    result = await kernel.invoke(plugin["Parrot"], count=2, user_message="I love parrots.", chat_history=chat_history)
    print(result)

